In [1]:
import sys,os
import numpy as np
import math
import pandas as pd
import joblib
from scipy.stats import entropy
from collections import Counter
import datetime
from itertools import repeat

In [9]:
data = pd.read_csv('C:/Users/ale/Documents/Tec/sleep_episodes/00001/sleep 1/sleep_1.csv')
#data['t'] = pd.to_datetime(data['t'])
#data['t'] = data['t'].astype('datetime64[m]')
data

,t,x,y,z,annotation0,arm_angle,change
0,132.043952,-0.196,-0.850,0.480,p_lie,28.822505,NaN
1,132.094000,-0.264,-0.846,0.572,p_lie,32.839298,4.016794
2,132.143856,-0.248,-0.798,0.558,p_lie,33.732823,0.893524
3,132.193904,-0.282,-0.752,0.542,p_lie,34.013614,0.280791
4,132.243952,-0.122,-0.858,0.578,p_lie,33.701348,-0.312265
...,...,...,...,...,...,...,...
1374,407.858904,0.464,-0.976,1.054,p_lie,44.283900,41.008066
1375,407.908952,-0.808,-0.360,1.614,p_lie,61.274585,16.990684
1376,407.959000,0.050,-0.324,0.752,p_lie,66.445146,5.170562
1377,408.008856,0.262,-1.002,0.758,p_lie,36.199696,-30.245451


In [10]:
# Get Euclidean Norm minus One
def get_ENMO(x,y,z):
    enorm = np.sqrt(x*x + y*y + z*z)
    ENMO = np.maximum(enorm-1.0, 0.0)
    return ENMO

# Get tilt angles
def get_tilt_angles(x,y,z):
    angle_x = np.arctan2(x, np.sqrt(y*y + z*z)) * 180.0/math.pi
    angle_y = np.arctan2(y, np.sqrt(x*x + z*z)) * 180.0/math.pi
    angle_z = np.arctan2(z, np.sqrt(x*x + y*y)) * 180.0/math.pi
    return angle_x, angle_y, angle_z

# Get Locomotor Inactivity During Sleep   
def get_LIDS(timestamp, ENMO):
    df = pd.concat((timestamp, pd.Series(ENMO)), axis=1)
    df.columns = ['timestamp','ENMO']
    df.set_index('timestamp', inplace=True)
    df['ENMO_sub'] = np.where(ENMO < 0.02, 0, ENMO-0.02) # assuming ENMO is in g
    # 10-minute rolling sum
    ENMO_sub_smooth = df['ENMO_sub'].rolling('600s').sum()
    df['LIDS_unfiltered'] = 100.0 / (ENMO_sub_smooth + 1.0)
    # 30-minute rolling average
    LIDS = df['LIDS_unfiltered'].rolling('1800s').mean().values
    return LIDS
  
def mad(data):
    out = np.mean(np.absolute(data - np.mean(data)))
    return out  

def compute_entropy(data, bins=20):
    bins = int(bins)
    hist, bin_edges = np.histogram(data, bins=bins)
    p = hist/float(hist.sum())
    ent = entropy(p)
    return ent 

# Get difference of feature with respect to prev or next interval
def get_diff_feat(feature, direction, time_diff, time_interval=30):
    window = int(time_diff / float(time_interval))
    mean_feature = feature.rolling(window).mean().values
    feature = feature.values.reshape(-1,1)
    mean_feature = mean_feature.reshape(-1,1)
    diff = np.zeros(feature.shape)
    if direction == 'prev':
        # Compute mean for border conditions
        for i in range(0,window-1):
            mean_feature[i] = np.mean(feature[:i+1])
    else:
        # Shift mean feature by window
        mean_feature = np.vstack((mean_feature[window-1:].reshape(-1,1),\
                              mean_feature[:window-1].reshape(-1,1)))
    # Compute mean for border conditions
        for i in range(len(feature)-window+1,len(feature)):
            mean_feature[i] = np.mean(feature[i:])
    if direction == 'prev':
        diff[1:] = feature[1:] - mean_feature[:-1]
    else:
        diff[:-1] = mean_feature[1:] - feature[:-1]

    return diff.reshape(-1,)
    
# Aggregate statistics of features over a given time interval
def get_stats(timestamp, feature, time_interval):
    feat_df = pd.DataFrame(data={'t':timestamp, 'feature':feature})
    feat_df.set_index('t', inplace=True)
    feat_mean = feat_df.resample(str(time_interval)+'S').mean()
    feat_std = feat_df.resample(str(time_interval)+'S').std()
    feat_min = feat_df.resample(str(time_interval)+'S').min()
    feat_max = feat_df.resample(str(time_interval)+'S').max()
    feat_range = feat_max['feature'] - feat_min['feature']
    feat_mad = feat_df.resample(str(time_interval)+'S').apply(mad)
    feat_ent1 = feat_df.resample(str(time_interval)+'S')\
                              .apply(compute_entropy, bins=20)
    feat_ent2 = feat_df.resample(str(time_interval)+'S')\
                              .apply(compute_entropy, bins=200)
    feat_prev30diff = get_diff_feat(feat_mean['feature'], 'prev', 30, time_interval)
    feat_next30diff = get_diff_feat(feat_mean['feature'], 'next', 30, time_interval)
    feat_prev60diff = get_diff_feat(feat_mean['feature'], 'prev', 60, time_interval)
    feat_next60diff = get_diff_feat(feat_mean['feature'], 'next', 60, time_interval)
    feat_prev120diff = get_diff_feat(feat_mean['feature'], 'prev', 120, time_interval)
    feat_next120diff = get_diff_feat(feat_mean['feature'], 'next', 120, time_interval)
    stats = np.vstack((feat_mean['feature'], feat_std['feature'], feat_range, 
                     feat_mad['feature'], feat_ent1['feature'], feat_ent2['feature'],
                     feat_prev30diff, feat_next30diff, feat_prev60diff, 
                     feat_next60diff, feat_prev120diff, feat_next120diff)).T

    # return index and stats
    return np.array(feat_mean.index.values, dtype='str'), stats

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def compute_features(data, time_interval):
    df = pd.DataFrame(data, columns=['t','x','y','z'])
    df['t'] = pd.to_datetime(df['t'], unit='s')

    x = np.array(df['x'])
    y = np.array(df['y'])
    z = np.array(df['z'])
    timestamp = pd.Series(df['t'])

    # Perform flipping x and y axes to ensure standard orientation
    # For correct orientation, x-angle should be mostly negative
    # So, if median x-angle is positive, flip both x and y axes
    # Ref: https://github.com/wadpac/hsmm4acc/blob/524743744068e83f468a4e217dde745048a625fd/UKMovementSensing/prepacc.py
    angx = np.arctan2(x, np.sqrt(y*y + z*z)) * 180.0/math.pi
    if np.median(angx) > 0:
        x *= -1
        y *= -1

    ENMO = get_ENMO(x,y,z)
    angle_x, angle_y, angle_z = get_tilt_angles(x,y,z)
    LIDS = get_LIDS(timestamp, ENMO)

    _, ENMO_stats = get_stats(df['t'], ENMO, time_interval)
    _, angle_z_stats = get_stats(df['t'], angle_z, time_interval)
    timestamp_agg, LIDS_stats = get_stats(df['t'], LIDS, time_interval)
    feat = np.hstack((ENMO_stats, angle_z_stats, LIDS_stats))

    return feat

In [12]:
def get_sleep_stage(data, time_interval, modeldir, mode):
    if mode == 'binary':
        states = ['Wake', 'Sleep']
    else: # multiclass
        states = ['Wake', 'NREM 1', 'NREM 2', 'NREM 3', 'REM']

    df = pd.DataFrame(data)
    df['t'] = pd.to_datetime(df['t'], unit='m')
    # Compute features
    feat = compute_features(df, time_interval)
    N = feat.shape[0]

    # Load nonwear models from given path and make predictions
    model_files = os.listdir(modeldir)
    nonwear_files = [fname for fname in model_files if 'nonwear' in fname]
    nfolds = len(nonwear_files)

    nw_pred = None
    for fold,fname in enumerate(nonwear_files):
        print('Predicting nonwear with model ' + str(fold+1))
        scaler, cv_clf = joblib.load(os.path.join(modeldir, fname))
        feat_sc = scaler.transform(feat)
        feat_sc = np.nan_to_num(feat_sc)
        fold_nw_pred = cv_clf.predict_proba(feat_sc)
        if fold == 0:
            nw_pred = fold_nw_pred
        else:
            nw_pred = nw_pred + fold_nw_pred
    nw_pred = nw_pred/float(nfolds)
    nw_pred = np.argmax(nw_pred, axis=1)

    # Load models from given path and make predictions for given mode
    model_files = [fname for fname in model_files if mode in fname]
    nfolds = len(model_files)

    y_pred = None
    for fold,fname in enumerate(model_files):
        print('Predicting sleep states with model ' + str(fold+1))
        scaler, cv_clf = joblib.load(os.path.join(modeldir, fname))
        feat_sc = scaler.transform(feat)
        feat_sc = np.nan_to_num(feat_sc)
        fold_y_pred = cv_clf.predict_proba(feat_sc)
        if fold == 0:
            y_pred = fold_y_pred
        else:
            y_pred = y_pred + fold_y_pred
    y_pred = y_pred/float(nfolds)
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = np.array([states[i] for i in y_pred])

    # Merge results of nonwear and sleep classification
    results = np.array(['Nonwear']*N)
    results[nw_pred == 0] = y_pred[nw_pred == 0]
    results = list(results)

    #sys.stdout.flush()
  
    return results

In [25]:
modeldir = 'C:/Users/ale/Documents/Tec/forest_model/'
time_interval = 900
print(time_interval)

900


In [26]:
results = get_sleep_stage(data, time_interval, modeldir, 'binary')

<ipython-input-10-b119b8a5e9bb>:34: RuntimeWarning: invalid value encountered in true_divide
  p = hist/float(hist.sum())


Predicting nonwear with model 1
Predicting nonwear with model 2
Predicting nonwear with model 3
Predicting nonwear with model 4
Predicting nonwear with model 5
Predicting sleep states with model 1
Predicting sleep states with model 2
Predicting sleep states with model 3
Predicting sleep states with model 4
Predicting sleep states with model 5


In [27]:
print(len(results))

20


In [28]:
print(results)

['Sleep', 'Sleep', 'Nonwear', 'Nonwear', 'Nonwear', 'Nonwear', 'Nonwear', 'Nonwear', 'Nonwear', 'Sleep', 'Sleep', 'Nonwear', 'Nonwear', 'Sleep', 'Sleep', 'Nonwear', 'Nonwear', 'Sleep', 'Sleep', 'Sleep']


In [18]:
print(data['t'].iloc[-1]-data['t'].iloc[0])

0 days 04:36:00.897120001
